In [7]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import numpy as np

pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_pickle('data_raw.pickle')
df.head(3)

,tconst,primaryTitle,averageRating,vote_count,vote_average,revenue,budget,num_genres,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western,inconnu,ratio_votes,director_x,director_fact,originalTitle,startYear,genres_x,runtime,overview,cast,director_y,original_language,poster_path
0,tt0499549,Avatar,7.9,29045,7.600,2923706026,237000000,3,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3676.582278,James Cameron,0,Avatar,2009,"Action,Adventure,Fantasy",162,"In the 22nd century, a paraplegic Marine is di...",Sam Worthington,James Cameron,anglais,/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg
1,tt4154796,Avengers: Endgame,8.4,23091,8.267,2794731755,356000000,3,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2748.928571,"Action, Adventure & Drama",1,Avengers: Endgame,2019,"Action,Adventure,Drama",181,After the devastating events of Avengers: Infi...,"Robert Downey Jr., Chris Evans",Inconnu,anglais,/or06FN3Dka5tukK1e9sl16pB3iy.jpg
2,tt1630029,Avatar: The Way of Water,7.6,7846,7.707,2319331580,460000000,3,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1032.368421,James Cameron,0,Avatar: The Way of Water,2022,"Action,Adventure,Fantasy",192,Set more than a decade after the events of the...,Sam Worthington,James Cameron,anglais,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg


In [6]:
df.loc[0,'poster_path']

'/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg'

In [9]:
table_totale_brute = pd.read_pickle('data_raw.pickle')

In [10]:
def filtrer_table(table_totale_brute):
    # GENRE
    # Convertir et formater la colonne 'genres_x'
    table_totale_brute['genres_x'] = table_totale_brute['genres_x'].astype(str).str.lower()
    # Créer une copie du DataFrame pour éviter de modifier l'original
    genre = table_totale_brute.copy()
    # Diviser la colonne 'genres_x' en listes
    genre['genres_x'] = genre['genres_x'].apply(lambda x: x.split(','))
    # Explosion du DataFrame pour traiter les listes de genres
    genre_1 = genre.explode('genres_x')
    # Remplacer les valeurs NaN dans la colonne 'genres_x' par 'inconnu'
    genre_1['genres_x'].replace('nan', 'inconnu', inplace= True)
    # Obtenir la liste unique des genres
    list_genres = genre_1['genres_x'].unique()
    # Saisie manuelle du genre par l'utilisateur
    genre_choisi = input(f"Choisissez un genre parmi {list_genres} (laissez vide pour tous): ")
    # Filtrer le DataFrame en fonction du genre choisi
    if genre_choisi:
        table_totale_brute = genre_1[genre_1['genres_x'] == genre_choisi]
    # ANNÉE
    # Saisie manuelle de l'année par l'utilisateur
    annee_choisie = input("Entrez une année (laissez vide pour tous) : ")
    if annee_choisie:
        # Convertir la colonne 'release_date' en datetime
        table_totale_brute['release_date'] = pd.to_datetime(table_totale_brute['release_date'], format='%Y-%m-%d')
        # Filtrer le DataFrame en fonction de l'année choisie
        table_totale_brute = table_totale_brute[table_totale_brute['release_date'].dt.year == int(annee_choisie)]
    # DURÉE
    # Trier le DataFrame par la colonne 'runtime'
    table_totale_brute = table_totale_brute.sort_values(by='runtime')
    # Afficher les classes de durée disponibles
    classes_de_duree = ['0-60 min', '60-120 min', 'au-delà de 120 min']
    print("Classes de durée disponibles :", classes_de_duree)
    # Saisie manuelle de la classe de durée par l'utilisateur
    duree_classe = input("Choisissez une classe de durée (laissez vide pour tous) : ")
    if duree_classe:
        # Filtrer le DataFrame en fonction de la classe de durée saisie
        if duree_classe == '0-60 min':
            table_totale_brute = table_totale_brute[table_totale_brute['runtime'] <= 60]
        elif duree_classe == '60-120 min':
            table_totale_brute = table_totale_brute[(table_totale_brute['runtime'] > 60) & (table_totale_brute['runtime'] <= 120)]
        elif duree_classe == 'au-delà de 120 min':
            table_totale_brute = table_totale_brute[table_totale_brute['runtime'] > 120]
    # ACTEUR/RÉALISATEUR
    # Choix de l'acteur
    nom_acteur_choisi = input("Choisissez un nom d'acteur (laissez vide pour tous) : ").lower()
    if nom_acteur_choisi:
        # Filtrer les données pour l'acteur choisi dans la colonne 'cast'
        table_totale_brute = table_totale_brute[table_totale_brute['cast'].str.lower().str.contains(nom_acteur_choisi, na=False)]
    # Choix du réalisateur
    realisateur_choisi = input("Choisissez un réalisateur (laissez vide pour tous) : ").lower()
    if realisateur_choisi:
        # Filtrer les données pour le réalisateur choisi dans la colonne 'directors'
        table_totale_brute = table_totale_brute[table_totale_brute['director'].str.lower().str.contains(realisateur_choisi, na=False)]
    # NOTE
    note_choisie = input("Choisissez une note (laissez vide pour tous) : ")  # faire des classes de valeurs
    if note_choisie:
        # la colonne 'vote_average' a le même type que la valeur entrée
        table_totale_brute['vote_average'] = table_totale_brute['vote_average'].astype(float)
        # Filtrer les données pour la note choisie
        table_totale_brute = table_totale_brute[table_totale_brute['vote_average'] == float(note_choisie)]
    # LANGUE
    # Saisie manuelle de la langue par l'utilisateur
    langue_choisie = input(f"Choisissez une langue entre (laissez vide pour tous) :   ")
    if langue_choisie:
        table_totale_brute = table_totale_brute.loc[table_totale_brute['original_language'] == langue_choisie]
    return table_totale_brute
# Utilisation de la fonction
resultat = filtrer_table(table_totale_brute)
# Afficher le DataFrame résultant
display(resultat)

Classes de durée disponibles : ['0-60 min', '60-120 min', 'au-delà de 120 min']


,tconst,primaryTitle,averageRating,vote_count,vote_average,revenue,budget,num_genres,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western,inconnu,ratio_votes,director_x,director_fact,originalTitle,startYear,genres_x,runtime,overview,original_language,poster_path,language_fact,Director,cast,actor_1,actor_2,actor_3,actor_4,actor_1_fact,actor_2_fact,actor_3_fact,actor_4_fact
168276,tt0138756,Romany Love,6.013641,0,0.000,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.000000,Fred Paul,51517,Romany Love,1931,romance,58,A musical film directed by Fred Paul,anglais,vide.png,0,Fred Paul,"Roy Travers, Esmond Knight, Florence McHugh, R...",Roy Travers,Esmond Knight,Florence McHugh,Rita Cave,57752,59478,80904,92475
169627,tt0173942,Jeep-Herders,6.700000,0,0.000,0,0,3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000000,Action,137,Jeep-Herders,1945,romance,58,A wildcat oil outfit is seeking to take over t...,anglais,/xZ7EKoV93C31W29LiWAykjgKQ1S.jpg,0,Harvey Parry,"Steve Clark, June Carlson, John Daheim, Pat Mi...",Steve Clark,June Carlson,John Daheim,Pat Michaels,58085,58197,81550,93246
123869,tt0028662,A Bride for Henry,5.600000,5,5.800,0,0,3,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.892857,William Nigh,4579,A Bride for Henry,1937,romance,58,On the day of her wedding a young woman's fian...,anglais,/sPoBKRJDjzJuYDtKbRZFBE04V7v.jpg,0,William Nigh,"Betty Ross Clarke, Claudia Dell, Henry Molliso...",Betty Ross Clarke,Claudia Dell,Henry Mollison,Anne Nagel,48346,57300,64223,69934
123717,tt0028270,Smartest Girl in Town,6.400000,3,5.333,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.468750,Joseph Santley,4463,Smartest Girl in Town,1936,romance,58,A girl in search of a rich husband mistakes a ...,anglais,/vlw5sAg9ejanDbk2pT7XgDM4WN9.jpg,0,Joseph Santley,"Eric Blore, Helen Broderick, Gene Raymond, Ann...",Eric Blore,Helen Broderick,Gene Raymond,Ann Sothern,48043,56779,64120,7203
130691,tt0026489,Hong Kong Nights,4.700000,2,3.500,0,0,3,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.425532,E. Mason Hopper,51300,Hong Kong Nights,1935,romance,58,Customs agents track a ring of arms smugglers ...,anglais,/f4GXQ52i98zS3ES05VBQPwHxHN5.jpg,0,E. Mason Hopper,"Tom London, Cornelius Keefe, Tom Keene, Warren...",Tom London,Cornelius Keefe,Tom Keene,Warren Hymer,47924,59547,64678,72894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101782,tt0072021,The Deluge,7.700000,57,7.228,0,0,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,7.402597,Jerzy Hoffman,12782,Potop,1974,romance,287,"During the Swedish invasion of Poland, the bra...",polonais,/vCariYegIocnhtwOegdRMByCQUB.jpg,25,Jerzy Hoffman,"Tadeusz Lomnicki, Kazimierz Wichniarz, Daniel ...",Tadeusz Lomnicki,Kazimierz Wichniarz,Daniel Olbrychski,Malgorzata Braunek,42258,48828,53435,57514
134842,tt0051082,Quiet Flows the Don,7.700000,19,6.500,0,0,3,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,2.467532,Sergey Gerasimov,45993,Tikhiy Don,1957,romance,340,Based on the novel of the same name by Mikhail...,russe,/dWHkHy3UBnXQiPJErQVz3AtVUzu.jpg,16,Sergey Gerasimov,"Zinaida Kirienko, Elina Bystritskaya, Pyotr Gl...",Zinaida Kirienko,Elina Bystritskaya,Pyotr Glebov,Daniil Ilchenko,43252,60533,68107,75865
112645,tt0095530,Little Dorrit,7.200000,11,6.400,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.527778,Christine Edzard,48258,Little Dorrit,1987,romance,357,A drama based on the novel by Charles Dickens ...,anglais,/61uclYiDIhkbV3o8XsjqUK0jbXD.jpg,0,Christine Edzard,"Max Wall, Joan Greenwood, Alec Guinness, Derek...",Max Wall,Joan Greenwood,Alec Guinness,Derek Jacobi,45327,49244,1431,57152
9663,tt0346336,The Best of Youth,8.500000,461,8.061,2693053,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,